In [ ]:
from nn.config import MODEL_LOCATION
from nn.nnModel import getModel
from training.nnTrainingSession import NNTrainingSession

model = getModel(MODEL_LOCATION)
session = NNTrainingSession(model)

In [ ]:
import re
import shutil
import os
import time
from os.path import isfile, join
from subprocess import Popen

from training.nnTrainingFile import createTrainingFiles
from nn.config import MODEL_NAME, MODEL_LOCATION, GAME_SOURCE_DIR, GAME_ARCHIVE_DIR_PREFIX, MODEL_ARCHIVE_DIR

botProcess = None

def createDirectoryIfNecessary(d):
    if not os.path.exists(d):
        os.makedirs(d)

def createDirectoriesIfNecessary():
    createDirectoryIfNecessary(GAME_SOURCE_DIR)
    createDirectoryIfNecessary(GAME_ARCHIVE_DIR_PREFIX)
    createDirectoryIfNecessary(MODEL_ARCHIVE_DIR)

def restartBots():
    botBasePath = r"C:\Users\Evan Sparano\Documents\GitHub\galcon-alphazero\gbots"
    botProcess = Popen(botBasePath + "\launch4Bots.bat", cwd=botBasePath)

def killBots():
    # TODO: this only kills one bot process? not all of them?
    if botProcess != None:
        os.kill(botProcess.pid, signal.SIGTERM)
    
def archiveGames(iterationNum):
    archiveDir = f"{GAME_ARCHIVE_DIR_PREFIX}/Iter{iterationNum}"
    os.makedirs(archiveDir)

    files = os.listdir(GAME_SOURCE_DIR)
    for f in files:
        fullFilePath = f"{GAME_SOURCE_DIR}/{f}"
        if 'npz' in f:
            shutil.move(fullFilePath, archiveDir)
        else:
            os.remove(fullFilePath)

def getCurrentIteration():
    onlyFiles = [f for f in os.listdir(MODEL_ARCHIVE_DIR) if isfile(join(MODEL_ARCHIVE_DIR, f)) and 'iter' in f]
    iters = [f.split('.')[1] for f in onlyFiles]
    return max([ int(re.findall(r'\d+', iteration)[0]) for iteration in iters], default=-1) + 1

def trainModel():
    # 10 loops because only a 10th of the data is actually sampled per loop
    # -1 numGamesPerLoop to use all games
    session.doTrain(num_loops=10, numGamesPerLoop=-1, numEpochs=1)
    print("trained the model!")

def nextIteration():
    # TODO: this isn't actually killing the bots, so the bots keep generating unused games while model is training
#     killBots()
    
    prevIterNum = getCurrentIteration()
    print(f"FINISHING ITERATION: {prevIterNum}")
    model.save(MODEL_ARCHIVE_DIR + f"/{MODEL_NAME}.iter{prevIterNum}.model")
    
    trainModel()
    model.save(MODEL_LOCATION)
    
    archiveGames(prevIterNum)
    
    restartBots()
    
def startTrainingLoop(maxIterations=3, numFilesPerIteration=10):
    createDirectoriesIfNecessary()
    restartBots()
    
    while getCurrentIteration() < maxIterations:
        time.sleep(10)
        createTrainingFiles()
        files = os.listdir(GAME_SOURCE_DIR)
        # multiply by two because of pickle and npz files
        if len(files) >= numFilesPerIteration * 2:
            nextIteration()
            
    killBots()
    

In [ ]:
startTrainingLoop(10000)

In [ ]:
from training.nnTrainingGraph import plotNNTrainingSession

plotNNTrainingSession(session)